# Segmenting and Clustering Neighborhoods in Toronto - Final Assessment (Part II)

## 1. Import all libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

<a id='item1'></a>

## 2. Load the dataset of Toronto Neighborhods

Load the dataset of Toront neighborhoods

In [10]:
#Using BeautifulSoup library
from bs4 import BeautifulSoup
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))

#Convert to the data to dataframe
neighborhood=pd.DataFrame(df[0])

#delete the cells with no assigned
neighborhood['Borough'].loc[neighborhood['Borough'] == 'Not assigned'] = neighborhood['Postal Code']

#Look the number of rows of the dataframe
neighborhood.shape

neighborhood

,Postal Code,Borough,Neighborhood
0,M1A,M1A,Not assigned
1,M2A,M2A,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,M8A,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


## 3. Install library geocoder and get latitude and longitude

In [11]:
#!conda install -c conda-forge geocoder
import geocoder # import geocoder

#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
print(neighborhoods.shape)
print(neighborhoods.head)

# getting the new dataframe with latitude y longitude
for ind in range(len(neighborhood)):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(neighborhood.loc[ind, "Postal Code"]))
    neighborhoods = neighborhoods.append({'Postal Code' :neighborhood.loc[ind, "Postal Code"],   
                                          'Borough' :neighborhood.loc[ind, 'Borough'],
                                          'Neighborhood' :neighborhood.loc[ind, 'Neighborhood'],
                                          'Latitude' :g.lat,
                                          'Longitude' :g.lng
                                         }, ignore_index=True)

        
print(neighborhoods.shape)
print(neighborhoods.head)


(0, 5)
<bound method NDFrame.head of Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood, Latitude, Longitude]
Index: []>


Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


(180, 5)
<bound method NDFrame.head of     Postal Code           Borough  \
0           M1A               M1A   
1           M2A               M2A   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
7           M8A               M8A   
8           M9A         Etobicoke   
9           M1B       Scarborough   
10          M2B               M2B   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
15          M7B               M7B   
16          M8B               M8B   
17          M9B         Etobicoke   
18          M1C       Scarborough   
19          M2C               M2C   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
24          M7C               M7C   